# SceneXplain and geospatial data

## 1. Download data

Run `get_data.sh` to download, extract and convert the dataset

## 2. Set options below

In [23]:
# SceneXplain options
FEATURES = ['json', 'high_quality']
ALGO = 'Flash'

# Dataset options
DATASET = "./data/UCMerced_LandUse/UCMerced_LandUse"
MAX_COUNT = 100

In [ ]:
!pip install -r requirements.txt

In [6]:
from dotenv import load_dotenv
import glob
import json
import os
import base64
import http

load_dotenv()

SCENEX_SECRET = os.getenv('SCENEX_SECRET')

In [20]:
from random import sample

image_folder = f'{DATASET}/Images'
image_files = glob.glob(f'{image_folder}/**/*.jpg')
shuffled_files = sample(image_files, MAX_COUNT)

In [8]:
# Create JSON schema - since each dataset may have different tags, we have to create it dynamically
category_dirs = os.listdir(image_folder)

with open('base_schema.json') as file:
    schema = json.loads(file.read())
    schema['properties']['category']['enum'] = category_dirs

In [9]:
headers = {
    "x-api-key": f"token {SCENEX_SECRET}",
    "content-type": "application/json",
}

def image_to_data_uri(file_path):
    with open(file_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
        return f"data:image/jpeg;base64,{encoded_image}"


def process_image(filename, schema, features=FEATURES):
    print(f'Processing {filename}')
    data = {
        "data": [
            {"image": image_to_data_uri(filename), "algorithm": ALGO, "features": features, "json_schema": json.dumps(schema)},
        ]
    }
    
    connection = http.client.HTTPSConnection("api.scenex.jina.ai")
    connection.request("POST", "/v1/describe", json.dumps(data), headers)
    response = connection.getresponse()
    
    response_data = response.read().decode("utf-8")
    response_json = json.loads(response_data)
    
    connection.close()

    return response_json

In [8]:
# def batch_process(image_list, schema, features=FEATURES):
#     data = {}
#     data['data'] = []
#     categories = []
#     for image_filename in image_list:
#         print(f"Processing {image_filename}")
#         category = image_filename.split("/")[-2]
#         categories.append(category)
#         data['data'].append(
#             {"image": image_to_data_uri(image_filename), "algorithm": ALGO, "features": features, "json_schema": json.dumps(schema)}
#         )

#     headers = {
#         "x-api-key": f"token {SCENEX_SECRET}",
#         "content-type": "application/json",
#     }

#     print("Sending payload")
#     connection = http.client.HTTPSConnection("api.scenex.jina.ai")
#     connection.request("POST", "/v1/describe", json.dumps(data), headers)
#     response = connection.getresponse()
    
#     response_data = response.read().decode("utf-8")
#     response_json = json.loads(response_data)
    
#     connection.close()

#     return response_json

In [24]:
successes = []
fails = []
errors = []

for filename in shuffled_files:
    category_name = filename.split('/')[-2]
    image_data = process_image(filename=filename, schema=schema, features=FEATURES)
    try:
        scenex_category = json.loads(image_data['result'][0]['i18n']['en'])['category'][0]
    except:
        scenex_category = "error processing"
        
    data = {
        "image_path": filename,
        "target_category": category_name,
        "scenex_category": scenex_category
    }

    if data['target_category'] == data['scenex_category']:
        data['match'] = True
        successes.append(data)
        print("\t✅ Successful match!")
    elif data['scenex_category'] == 'error processing':
        errors.append(data)
        print("\t😭 Error")
        print(image_data)
    else:
        data['match'] = False
        fails.append(data)
        print(f"\t❌ Failed match! (Identified {data['target_category']} as {data['scenex_category']})")

Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/storagetanks/storagetanks69.jpg
	❌ Failed match! (Identified storagetanks as buildings)
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/beach/beach45.jpg
	✅ Successful match!
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/storagetanks/storagetanks41.jpg
	✅ Successful match!
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/buildings/buildings49.jpg
	✅ Successful match!
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/mediumresidential/mediumresidential43.jpg
	✅ Successful match!
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/intersection/intersection77.jpg
	✅ Successful match!
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/baseballdiamond/baseballdiamond74.jpg
	✅ Successful match!
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/Images/airplane/airplane55.jpg
	❌ Failed match! (Identified airplane as runway)
Processing ./data/UCMerced_LandUse/UCMerced_LandUse/I

In [26]:
# what percent did we get right?
len(successes)/(len(successes)+len(fails))

0.7448979591836735

## Testing results

What success rate did we get?

- **Flash**: 75%

- **Jelly**: 61%
- **Jelly/High quality**: 66% 